## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-22-05-47-14 +0000,nyt,Vehicle Crashes Into Security Gate Outside the...,https://www.nytimes.com/2025/10/21/us/white-ho...
1,2025-10-22-05-36-13 +0000,nypost,Customer shoots dead alleged creep who groped ...,https://nypost.com/2025/10/22/us-news/la-custo...
2,2025-10-22-05-17-52 +0000,nyt,U.S. Marshal and Mexican Immigrant Are Shot Du...,https://www.nytimes.com/2025/10/21/us/us-marsh...
3,2025-10-22-05-16-00 +0000,wsj,Japan’s Exports Rebound From Months of Decline...,https://www.wsj.com/economy/japans-exports-reb...
4,2025-10-22-05-11-28 +0000,bbc,Gaza health crisis will last for 'generations'...,https://www.bbc.com/news/articles/cnvezzdnmrno...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,50
6,house,18
189,up,15
5,white,15
35,gaza,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
281,2025-10-21-11-43-48 +0000,cbc,White House demolition work begins as Trump pu...,https://www.cbc.ca/news/world/us-white-house-d...,106
178,2025-10-21-18-44-04 +0000,bbc,From bowling alley to ballroom - Trump isn't t...,https://www.bbc.com/news/articles/cr7m2v80dy4o...,106
190,2025-10-21-17-55-57 +0000,nypost,Hillary mocked after blasting Trump’s White Ho...,https://nypost.com/2025/10/21/us-news/hillary-...,105
233,2025-10-21-15-54-55 +0000,nypost,White House rips report Trump could commute Di...,https://nypost.com/2025/10/21/us-news/white-ho...,99
34,2025-10-22-02-04-22 +0000,nyt,As Trump Moves Forward With White House Ballro...,https://www.nytimes.com/2025/10/21/arts/design...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
281,106,2025-10-21-11-43-48 +0000,cbc,White House demolition work begins as Trump pu...,https://www.cbc.ca/news/world/us-white-house-d...
26,75,2025-10-22-03-11-48 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
71,52,2025-10-21-23-38-00 +0000,wsj,Paul Ingrassia withdrew Tuesday from a Senate ...,https://www.wsj.com/politics/policy/paul-ingra...
249,45,2025-10-21-14-29-17 +0000,nypost,Japan’s parliament elects ultraconservative Sa...,https://nypost.com/2025/10/21/world-news/japan...
191,42,2025-10-21-17-44-24 +0000,nypost,Hamas released bodies of two more dead hostage...,https://nypost.com/2025/10/21/world-news/hamas...
219,42,2025-10-21-16-33-45 +0000,wapo,CNN’s future at stake as owner Warner Bros. Di...,https://www.washingtonpost.com/business/2025/1...
215,40,2025-10-21-16-39-19 +0000,nyt,France Sends Former President Nicolas Sarkozy ...,https://www.nytimes.com/2025/10/21/world/europ...
265,39,2025-10-21-13-32-06 +0000,nyt,"Suspect Charged in Paris Museum Heist (No, Not...",https://www.nytimes.com/2025/10/21/world/europ...
58,39,2025-10-22-00-24-39 +0000,nypost,Hunter Biden says watching Obama lead Joe off ...,https://nypost.com/2025/10/21/us-news/hunter-b...
210,37,2025-10-21-17-02-40 +0000,nypost,Top Trump adviser boldly predicts Democrats wi...,https://nypost.com/2025/10/21/us-news/top-trum...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
